### Interactive LLM

### Step 0 : Importing our Model from OLLAMA


In [15]:
import os
from langchain_community.llms.ollama import Ollama

# Load the locally hosted model
model_path = 'llama3.1'  # Ensure this path corresponds to the correct model name
llm = Ollama(model=model_path)

# Set the temperature parameter if needed
llm.temperature = 0

# Define a function to ask the LLM a question
def ask_llm(prompt):
    response = llm(prompt)
    return response

def ask_llm2(patient_conditions, current_node,prompt):
    # Integrate the prompt with patient conditions and current node
    full_prompt = f"\nPatient Conditions: {patient_conditions}\nCurrent Node: {current_node}\n{prompt}\n"
    response = llm(full_prompt)
    return response.strip()



In [10]:
name_of_graphs

['Patients with newly diagnosed or recurrent MBC',
 'MANAGEMENT OF ER-POSITIVE, HER2-NEGATIVE MBC',
 'FIRST- AND SECOND-LINE TREATMENT OF HER2-POSITIVE MBC',
 'THIRD-LINE AND BEYOND TREATMENT OF HER2-POSITIVE MBC',
 'MANAGEMENT OF mTNBC',
 'MANAGEMENT OF EARLY TNBC',
 'MANAGEMENT OF HER2-POSITIVE EBC',
 'ROLE OF ADJUVANT ET IN HR-POSITIVE EBC',
 'SYSTEMIC TREATMENT OF HR-POSITIVE, HER2-NEGATIVE EBC',
 'MANAGEMENT OF ALN INVOLVEMENT IN EBC',
 'Patients with a suspicion of OMD',
 'BRCA1 PV',
 'BRCA2 PV',
 'Diagnosis and staging of EBC',
 'Overview of EBC treatment']

### Step1 : Find the equivalent graph to this patient

In [16]:
def find_the_equivalent_graph_to_this_patient(name_of_graphs,patient_file_paragraph):
    prompt = f"""
    From this list of names of graphs :{' ,'.join(i for i in name_of_graphs)}, pick the graph that exactly matches any reference in this patient file {patient_file_paragraph}.
    Return **only** the exact name of the graph. If there is no matching graph, return 'idk'.
    **Do not provide explanations or any other information.**
    """

    graph_name = ask_llm(prompt)
    return graph_name

In [13]:
x="""A 62-year-old postmenopausal woman with ER-positive, HER2-negative metastatic breast cancer.
She has bone metastases but no visceral involvement.
She has been previously treated with an aromatase inhibitor (AI) and a CDK4/6 inhibitor."""
s=find_the_equivalent_graph_to_this_patient(name_of_graphs,x)


In [14]:
s

'MANAGEMENT OF ER-POSITIVE, HER2-NEGATIVE MBC'

### Step2:Convert to graph
MOCHKELT DATABASE ARCHITETURE INAJMOU INBADLOUHA 

In [4]:
def convert_to_graph(input_dict):
    # Initialize the graph dictionary
    graph = {}

    # Extract nodes and edges from the input dictionary
    nodes = input_dict['nodes']
    edges = input_dict['edges']
    
    # Create an entry for each node
    for node, description in nodes.items():
        graph[node] = []

    # Create edges based on the input
    for start, end, condition in edges:
        # Add the edge to the graph
        if start in graph:
            graph[start].append((end, condition))

    return graph

### Step 3: From paraph to a list extraction


In [5]:
def fromparaph_toalist_extraction(paragraph):
    prompt1 = f"""
        Extract the following key medical information from the given text into a structured list:
        - Age
        - Gender
        - Medical history
        - Diagnoses
        - Treatments
        - Medications
        - Relevant information for clinical decision-making

        Text: "{paragraph}"

        Output the information in a list format without labels or extra explanations:
        [Age, Gender, Medical history, Diagnoses, Treatments, Medications, Relevant information for clinical decision-making]

        **Response should include all relevant details in a simple list format as shown above.**
        """
    response = ask_llm(prompt1)
    #print("LLM Response:", response)
    #output list 7athra
    l=response[1:-1].split(',')
    l.append('')
    return l


### Step 4 : Parcour the graph and give the node or question if you need more info

In [6]:
def traverse_with_conditions(graph, start_node, conditions,path=[],graph_name="Breast Cancer Treatment Pathway"):
    path=[]
    current_node = start_node
    while(1):
        
        found=False
        print(current_node)
        x=[]
        suiv=[]
        for neighbor, edge_condition in graph[current_node]:
            

            if edge_condition in conditions:
                    path.append((current_node, neighbor))
                    current_node = neighbor
                    found = True
                    break   
            else:
                x.append(edge_condition)
                suiv.append(neighbor)


        if not found:
            if len(suiv)!=0:
                
                
                

                prompt = f"""
                    You are traversing a medical decision graph using Depth-First Search (DFS).
                    Nodes represent conditions, and edges represent possible transitions.

                    Current state:
                    - Graph: {graph}
                    - Current Node: '{current_node}'
                    - Neighbors: {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}

                    **Instructions:**
                    1. If the patient's conditions match any of the transition conditions exactly, respond with "nextnode:<name of the node>" for the correct node.
                    2. If no match is found or if more information is needed, ask for it with "question: Give me the status of <condition>".
                    3. Provide only one output, either "nextnode:<name of the node>" or "question:<specific question>".

                    **Rules:**
                    - Do not make assumptions. Proceed only if conditions match exactly.
                    - If unsure, always ask for more information.

                    
                    **Output ONLY:**
                     - If all conditions match exactly one node: "nextnode:<name of the node>"
                     - If additional information is needed to proceed: "question: {" or ".join([f'Give me {c} status?' for c in x])}"

                    **Do not provide explanations or any other information.**
                    """


                #print(prompt)
                
                
                
                response = ask_llm(prompt)
                print(response)
                ch=""
                if "nextnode:" in response:
                    ch=response[len("nextnode:"):].replace(" ", "")
                    path.append((current_node, ch))
                    current_node = ch
                    print(current_node,len(current_node))
                    found = True
                       
                else:
                    
                    print(response)
                    #print(f"taw netfehmou khater lezemik '{x}' condition w enty f {current_node}.")
                    break
            else:
                print("wselt node final:",current_node)
                break

    return path


### Final treatment output function !


In [7]:
import time
def Frompatient_file_paragraph_to_following_treatment(name_of_graphs,patient_file_paragraph):
    # Start time
    start_time = time.time()
    prompt = f"""
From this list of names of graphs {name_of_graphs}, pick the graph that exactly matches any reference in this patient file {patient_file_paragraph}.
Return **only** the exact name of the graph. If there is no matching graph, return 'idk'.
**Do not provide explanations or any other information.**
"""

    #graph_name = s
    # Your code here
    graph_name=find_the_equivalent_graph_to_this_patient(name_of_graphs,patient_file_paragraph)
    if graph_name=='idk':
        return False
    print("graph_name:", graph_name)
    graph=convert_to_graph(graphs[graph_name])
    print("converting graph output:\n",graph)
    patient_conditions=fromparaph_toalist_extraction(patient_file_paragraph)
    print("patient_conditions list",patient_conditions)
    # Start the traversal from node 'A'
    path_taken = traverse_with_conditions(graph, 'A', patient_conditions)

    # Output the path taken
    print("Path taken:")
    for step in path_taken:
        print(f"  {step[0]} -> {step[1]}")
    
    # End time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    return True

In [ ]:
#Some cases
#LLM Response: MANAGEMENT OF HR-POSITIVE, HER2-NEGATIVE MBC
#LLM Response: FIRST- AND SECOND-LINE TREATMENT OF HER2-POSITIVE MBC
#LLM Response: MANAGEMENT OF mTNBC
"""A 62-year-old postmenopausal woman with ER-positive, HER2-negative metastatic breast cancer.
She has bone metastases but no visceral involvement.
She has been previously treated with an aromatase inhibitor (AI) and a CDK4/6 inhibitor.
-----------------------------------------
A 48-year-old premenopausal woman with HER2-positive, ER-negative metastatic breast cancer. The cancer has metastasized to the liver and lungs. She has not been treated with any targeted therapy for HER2.
------------------------------------------
A 45-year-old woman with metastatic triple-negative breast cancer (mTNBC). She is PD-L1 positive and has a germline BRCA1 mutation. The cancer has spread to the lungs and bones.
"""


### Enter Patient File

In [8]:
patient_file_paragraph=input()

A 62-year-old postmenopausal woman with ER-positive, HER2-negative metastatic breast cancer. She has bone metastases but no visceral involvement. She has been previously treated with an aromatase inhibitor (AI) and a CDK4/6 inhibitor.


### import DB (graphs)

In [2]:
#1-identify the correspond graph
import ast

# Step 1: Read the content of the file
with open('Graphs.txt', 'r') as file:
    file_content = file.read()

# Step 2: Convert the string content to a dictionary
graphs = ast.literal_eval(file_content)

# Step 3: Now, 'graphs' is a dictionary that you can use
print(type(graphs))



<class 'dict'>


In [3]:
name_of_graphs=[]
for i in graphs:
    name_of_graphs.append(i)


In [11]:
name_of_graphs

['Patients with newly diagnosed or recurrent MBC',
 'MANAGEMENT OF ER-POSITIVE, HER2-NEGATIVE MBC',
 'FIRST- AND SECOND-LINE TREATMENT OF HER2-POSITIVE MBC',
 'THIRD-LINE AND BEYOND TREATMENT OF HER2-POSITIVE MBC',
 'MANAGEMENT OF mTNBC',
 'MANAGEMENT OF EARLY TNBC',
 'MANAGEMENT OF HER2-POSITIVE EBC',
 'ROLE OF ADJUVANT ET IN HR-POSITIVE EBC',
 'SYSTEMIC TREATMENT OF HR-POSITIVE, HER2-NEGATIVE EBC',
 'MANAGEMENT OF ALN INVOLVEMENT IN EBC',
 'Patients with a suspicion of OMD',
 'BRCA1 PV',
 'BRCA2 PV',
 'Diagnosis and staging of EBC',
 'Overview of EBC treatment']

In [14]:
Frompatient_file_paragraph_to_following_treatment(name_of_graphs,patient_file_paragraph)

graph_name: MANAGEMENT OF ER-POSITIVE, HER2-NEGATIVE MBC
converting graph output:
 {'A': [('B', ''), ('C', 'If imminent organ falure')], 'B': [('D', '')], 'C': [('B', 'PD or intolerable toxicities'), ('E', '')], 'D': [('I', ''), ('J', '')], 'E': [('D', '')], 'F': [('D', '')], 'I': [('K', ''), ('L', ''), ('M', ''), ('N', '')], 'J': [('P', '')], 'K': [('O', '')], 'L': [('O', '')], 'M': [('O', '')], 'N': [('O', '')], 'O': [('P', '')], 'P': [('Q', '')], 'Q': [('R', 'If HER2-0'), ('S', 'If HER2-Low')], 'R': [('T', '')], 'S': [('T', '')], 'T': []}
patient_conditions list ['62', ' Female', ' Postmenopausal', ' Metastatic breast cancer', ' Bone metastases', ' Aromatase inhibitor (AI)', ' CDK4/6 inhibitor', ' ER-positive', ' HER2-negative', '']
A
B
D
I
K
O
P
Q
question: Give me If HER2-0 status?
question: Give me If HER2-0 status?
Path taken:
  A -> B
  B -> D
  D -> I
  I -> K
  K -> O
  O -> P
  P -> Q
Elapsed time: 53.04077482223511 seconds


True

### To Test

In [20]:
patient_conditions = [
    "",
    "New diagnosis of metastatic breast cancer (MBC)", 
    "Biopsy confirms diagnosis", 
    
    "ER positive",  
    "HER2 Negative"
    
    
 
]
graph = {
    'A': [('B', 'New diagnosis of metastatic breast cancer (MBC)')],
    'B': [('C', 'Biopsy confirms diagnosis')],
    'C': [('D', 'ER positive, HER2 negative'), ('E', 'TNBC (Triple Negative Breast Cancer)')],
    'D': [('E', 'PIK3CA mutation negative'),('F', 'PIK3CA mutation positive')],
    'E': [],
    'F': [('G', 'Patient eligible for targeted therapy')],
    'G': []
}


In [26]:
path_taken = traverse_wit_conditions(graph, 'A', patient_conditions)

A
B
C
D
Give me PIK3CA mutation status ?


In [24]:
def traverse_wit_conditions(graph,x,patient_conditions):
    print("A")
    print("B")
    print("C")
    print("D")
    print("Give me PIK3CA mutation status ?")

### Some prompt if you want to learn

In [104]:
prompt =f"""
                        I am currently traversing a graph using the Depth-First Search (DFS) algorithm to evaluate specific conditions in a patient's medical file. 
                        The nodes represent specific conditions, and the edges represent possible transitions between these conditions.

                        The decision on which node to traverse next is critical because it directly impacts the treatment plan for the patient. Therefore, focus **only** on the conditions provided and the conditions on the edges of the current node.

                        I am currently at node '{current_node}', which represents '{graph[current_node]}'. 
                        The neighboring nodes are {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}.
                        Based on the patient's current conditions: {conditions}, suggest the next node to move to, or ask for any specific condition needed to make this decision.

                         **Output ONLY in the following format:**

                        - If conditions are met: "nextnode:<name of the node>"
                        - If additional information is required: "question: <question containing the condition(s) needed to determine the next node>"
                        """
prompt = f"""
                        I am currently traversing a graph using the Depth-First Search (DFS) algorithm to evaluate specific conditions in a patient's medical file. 
                        The nodes represent specific conditions, and the edges represent possible transitions between these conditions.

                        The decision on which node to traverse next is critical because it directly impacts the treatment plan for the patient. Therefore, it is essential that you focus **only** on the conditions provided and the conditions on the edges of the current node.

                        I am currently at node '{current_node}', which represents '{graph[current_node]}'. 
                        The neighboring nodes are {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}.

                        ### Important Instructions:
                        1. **Focus Exclusively on Conditions**: Only consider the provided conditions: {', '.join(conditions)} and the conditions on the edges connected to the current node. Ignore any other information or assumptions.
                        2. **Decision Making**:
                            - If one or more neighboring nodes' edge conditions match the provided conditions, respond with the exact format: 'NEXT_NODE:<name_of_node>'.
                            - If none of the neighboring nodes are suitable based on the edge conditions, or if you need more information to make a decision, respond with the exact format: 'REQUEST_INFO:<specific_question_or_data_needed>'.
                        3. **Formulating a Combined Question**: If additional information is required to make a decision, combine the necessary conditions and formulate a clear and concise question. For example, if the edge conditions for node C are: [('D', 'ER positive, HER2 negative'), ('E', 'TNBC (Triple Negative Breast Cancer')], your question should be:
                            - 'Give me ER and HER2 status? or TNBC status?'
                            - This question should guide the retrieval of the precise data needed to make an informed decision.
                        4. **Do Not Guess**: If you are uncertain or the conditions do not match exactly, request more information rather than guessing. The accuracy of this decision is crucial for the patient's treatment.

                        Your decision should be based **solely** on the provided conditions and the conditions of the edges from the current node. If needed, formulate a specific question that combines the conditions to request the necessary information.
                        """
###################################
prompt = f"""
                        I am currently traversing a graph using the Depth-First Search (DFS) algorithm to evaluate specific conditions in a patient's medical file. 
                        The nodes represent specific conditions, and the edges represent possible transitions between these conditions.

                        The decision on which node to traverse next is critical because it directly impacts the treatment plan for the patient. Therefore, it is essential that you focus **only** on the conditions provided and the conditions on the edges of the current node.

                        I am currently at node '{current_node}', which represents '{graph[current_node]}'. 
                        The neighboring nodes are {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}.

                        ### Important Instructions:
                        1. **Focus Exclusively on Conditions**: Only consider the provided conditions: {', '.join(conditions)} and the conditions on the edges connected to the current node. Ignore any other information or assumptions.
                        2. **Decision Making**:
                            - If one or more neighboring nodes' edge conditions match the provided conditions, respond with the exact format: 'NEXT_NODE:<name_of_node>'.
                            - If none of the neighboring nodes are suitable based on the edge conditions, or if you need more information to make a decision, respond with the exact format: 'REQUEST_INFO:<specific_question_or_data_needed>'.
                        3. **Do Not Guess**: If you are uncertain or the conditions do not match exactly, request more information rather than guessing. The accuracy of this decision is crucial for the patient's treatment.

                        Your decision should be based **solely** on the provided conditions and the conditions of the edges from the current node.
                        """
#########################################
prompt = f"""
                I am traversing a graph using Depth-First Search (DFS) to evaluate conditions in a patient's medical file.
                Nodes represent conditions, and edges represent possible transitions.
                The graph is: {graph}
                Currently at node '{current_node}'
                List the neighbors: {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}

                **All conditions required for a transition must be explicitly provided and satisfied before proceeding to the next node.**

                If any condition is missing or not explicitly provided, you **must** request more information before proceeding. Do not guess or infer. The accuracy of this decision is crucial for the patient's treatment.

                **Incorrect decisions may result in suboptimal treatment plans.**

                Formulate a specific question to request the missing information if needed.
                **Do Not Guess**: If you are uncertain or if even one condition does not match exactly, request more information rather than guessing.

                **Output ONLY:**

                - If all conditions match exactly one node: "nextnode:<name of the node>"
                - If additional information is needed to proceed: "question: {" or ".join([f'Give me {c} status?' for c in x])}"

                **Do not provide explanations or any other information.**
"""
##########################################
                prompt = f"""
                    You are traversing a medical decision graph using Depth-First Search (DFS).
                    Nodes represent conditions, and edges represent possible transitions.

                    Current state:
                    - Graph: {graph}
                    - Current Node: '{current_node}'
                    - Neighbors: {', '.join([f"{n} (condition: {c})" for n, c in zip(suiv, x)])}

                    **Instructions:**
                    1. Check if the patient's conditions match any of the transition conditions listed.
                    2. If a match is found, provide the next node in the format: "nextnode:<name of the node>".
                    3. If no match is found or if more information is needed, ask for it in the format: "question:<specific question>".

                    **Rules:**
                    - Do not make assumptions. Only proceed if conditions match exactly.
                    - If unsure, always ask for more information.

                    **Output ONLY:**
                     - If all conditions match exactly one node: "nextnode:<name of the node>"
                     - If additional information is needed to proceed: "question: {" or ".join([f'Give me {c} status?' for c in x])}"

                    **Do not provide explanations or any other information.**
                    """

In [ ]:
"""
THIRD-LINE AND BEYOND TREATMENT OF HER2-POSITIVE MBC":{
        "nodes": {
            "A": "Patients with HER2+ MBC\n 3rd-line treatment and beyond",
            "B": "Road of Active BMs",
            "C": "Road of No, unknown or stable BMs",
            "D": "Local intervention indicated.",
            "E": "Local intervention not indicated",
            "F": "Case of BMs beteween 1-10, favourable prognostic factors"
            "G": "Resction",
            "H": "SRT",
            "I": "Case of BMs < 10 , unfavourable prognostic factors",
            "J":"WBT",
            "K": "Tucatinib-capecitabine-trastuzumab",
            "L": "Trastuzumab deruxtecan",
            "M": "Tucatinib-capecitabine-trastuzumab or Trastuzumab deruxtecan+ or T-DM1#",
            "N": "Lapatinib-capecitabine Lapatinib-trastuzumab* Trastuzumab-ChT* Margetuximab-ChT*t Neratinib-ChT*+"
            }
            "edges": [
            ("A","B","Active BMs"),
            ("A","C","No, unknown or stable BMs"),
            ("B","D","Local intervention indicated"),
            ("B","E","Local intervention not indicated"),
            ("E","F","BMs value between 1 and 10 and favourable prognostic factors"),
            ("F","G","BMs value >= 3 or Located in or near critical areas of the brain or Significant neurological symptoms or Rapidly growing lesions or Good overall health and ability to tolerate surgery"),
            ("F","H","BMs value < 3 or Located in non-critical areas of the brain or  Minimal or no neurological symptoms or Slow-growing or stable lesions or Patient has comorbidities or poor performance status"),
            ("G","H",""),
            
            ("E","I","BMs value >10 and unfavourable prognostic factors")
            ("I","J",""),
            
            ("D","K",""),
            ("K","L","the disease progresses or if the first combination is not suitable."),
            ("L","N","")
            ("C","M",""),
            ("M","N","")
            
            ]

In [36]:
graph = {
    'A': [('B', 'Active BMs'), ('C', 'No, unknown or stable BMs')],
    'B': [('D', 'Local intervention indicated'), ('E', 'Local intervention not indicated')],
    'C': [('M', '')],
    'D': [('K', '')],
    'E': [('F', 'BMs value between 1 and 10 and favourable prognostic factors'), ('I', 'BMs value >10 and unfavourable prognostic factors')],
    'F': [('G', 'BMs value >= 3 or Located in or near critical areas of the brain or Significant neurological symptoms or Rapidly growing lesions or Good overall health and ability to tolerate surgery'),
          ('H', 'BMs value < 3 or Located in non-critical areas of the brain or Minimal or no neurological symptoms or Slow-growing or stable lesions or Patient has comorbidities or poor performance status')],
    'G': [('H', '')],
    'H': [],
    'I': [('J', '')],
    'J': [],
    'K': [('L', 'the disease progresses or if the first combination is not suitable.')],
    'L': [('N', '')],
    'M': [('N', '')],
    'N': []
}


In [28]:
patient="""The patient has HER2-positive metastatic breast cancer (MBC) and is undergoing 3rd-line treatment or beyond. The current status of the patient's brain metastases (BMs) is active. Local intervention for these BMs is not indicated. The BMs are evaluated to be between 1 and 10 and the patient has favorable prognostic factors. Given these conditions, the next step in the treatment plan is to assess the nature of the BMs further. Depending on whether the BMs meet specific criteria, the treatment may involve either resection if the BMs are located in or near critical areas of the brain with significant neurological symptoms or rapidly growing lesions, or stereotactic radiotherapy (SRT) if the BMs are in non-critical areas, minimal or stable, or if the patient has comorbidities or poor performance status. If local intervention were indicated, the treatment would involve Tucatinib-capecitabine-trastuzumab. If the BMs had been greater than 10 with unfavorable prognostic factors, the patient would have received whole brain radiotherapy (WBT). For patients with no, unknown, or stable BMs, treatment options include Tucatinib-capecitabine-trastuzumab, Trastuzumab deruxtecan, or T-DM1."""

In [29]:
l=fromparaph_toalist_extraction(patient)

In [32]:
print(l)

['HER2+ MBC', '3rd-line or beyond', 'local intervention indicated', 'active BMs', 'Number of BMs: 5', 'Prognostic Factors: Favorable', 'Age: 52', 'Patient Name: Jane Doe']


In [39]:
l=["HER2+ MBC","3rd-line or beyond","local intervention indicated","active BMs","Number of BMs: 5","Prognostic Factors: Favorable","Age: 52","Patient Name: Jane Doe"]
path_taken = traverse_with_conditions(graph, 'A', l)

# Output the path taken
print("Path taken:")
for step in path_taken:
    print(f"  {step[0]} -> {step[1]}")

A
question: Give me Active BMs status?
question: Give me Active BMs status?
Path taken:
